In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Eczema"
cohort = "GSE63741"

# Input paths
in_trait_dir = "../../input/GEO/Eczema"
in_cohort_dir = "../../input/GEO/Eczema/GSE63741"

# Output paths
out_data_file = "../../output/preprocess/Eczema/GSE63741.csv"
out_gene_data_file = "../../output/preprocess/Eczema/gene_data/GSE63741.csv"
out_clinical_data_file = "../../output/preprocess/Eczema/clinical_data/GSE63741.csv"
json_path = "../../output/preprocess/Eczema/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene Expression Analyses of Homo sapiens Inflammatory Skin Diseases"
!Series_summary	"Transcriptional profiling of Homo sapiens inflammatory skin diseases (whole skin biospies): Psoriasis (Pso), vs Atopic Dermatitis  (AD) vs Lichen planus (Li), vs Contact Eczema (KE), vs Healthy control (KO)"
!Series_summary	"In recent years, different genes and proteins have been highlighted as potential biomarkers for psoriasis, one of the most common inflammatory skin diseases worldwide. However, most of these markers are not psoriasis-specific but also found in other inflammatory disorders.  We performed an unsupervised cluster analysis of gene expression profiles in 150 psoriasis patients and other inflammatory skin diseases (atopic dermatitis, lichen planus, contact eczema, and healthy controls). We identified a cluster of IL-17/TNFα-associated genes specifically expressed in psoriasis, among which IL-36γ was the most outstanding marker. In subsequent immuno

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on background information, this dataset contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# From the background information, we can see this is a study comparing different skin conditions
# including Atopic Dermatitis (AD) which is a form of Eczema
# The dataset contains samples from patients with Eczema (Contact Eczema - KE) and other conditions

# For trait (Eczema), we need to infer from 'sample type' information
trait_row = 1  # The information about disease status is in row 1

# Age is not explicitly mentioned in the sample characteristics
age_row = None

# Gender is not explicitly mentioned in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert trait value to binary (0 or 1).
    1 if the sample is from an Eczema patient (AD - Atopic Dermatitis or KE - Contact Eczema)
    0 if the sample is from a non-Eczema patient or healthy control
    """
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    # Check if the value indicates Eczema
    if 'atopic dermatitis' in value or 'contact eczema' in value or 'ad' in value or 'ke' in value:
        return 1
    elif 'healthy' in value or 'control' in value or 'ko' in value or 'psoriasis' in value or 'lichen planus' in value or 'pso' in value or 'li' in value:
        return 0
    else:
        return None

# These functions are not needed as age and gender data are not available,
# but we'll define them as placeholders
def convert_age(value):
    return None

def convert_gender(value):
    return None

# 3. Save Metadata
# Conduct initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM1556392': [0.0], 'GSM1556393': [0.0], 'GSM1556394': [0.0], 'GSM1556395': [0.0], 'GSM1556396': [0.0], 'GSM1556397': [0.0], 'GSM1556398': [0.0], 'GSM1556399': [0.0], 'GSM1556400': [0.0], 'GSM1556401': [0.0], 'GSM1556402': [0.0], 'GSM1556403': [0.0], 'GSM1556404': [0.0], 'GSM1556405': [0.0], 'GSM1556406': [0.0], 'GSM1556407': [0.0], 'GSM1556408': [0.0], 'GSM1556409': [0.0], 'GSM1556410': [0.0], 'GSM1556411': [0.0], 'GSM1556412': [0.0], 'GSM1556413': [0.0], 'GSM1556414': [0.0], 'GSM1556415': [0.0], 'GSM1556416': [0.0], 'GSM1556417': [0.0], 'GSM1556418': [0.0], 'GSM1556419': [0.0], 'GSM1556420': [0.0], 'GSM1556421': [0.0], 'GSM1556422': [0.0], 'GSM1556423': [0.0], 'GSM1556424': [0.0], 'GSM1556425': [0.0], 'GSM1556426': [0.0], 'GSM1556427': [0.0], 'GSM1556428': [0.0], 'GSM1556429': [0.0], 'GSM1556430': [0.0], 'GSM1556431': [0.0], 'GSM1556432': [0.0], 'GSM1556433': [0.0], 'GSM1556434': [0.0], 'GSM1556435': [0.0], 'GSM1556436': [0.0], 'GSM1556437': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Eczema/GSE63741/GSE63741_series_matrix.txt.gz
Gene data shape: (1542, 150)
First 20 gene/probe identifiers:
Index(['3', '5', '16', '18', '20', '33', '35', '37', '39', '43', '47', '49',
       '55', '57', '59', '61', '67', '71', '73', '81'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers in the data appear to be numeric identifiers (e.g., '3', '5', '16', etc.)
# These are not standard human gene symbols (which would look like BRCA1, TP53, IL6, etc.)
# These appear to be probe IDs or some other numeric identifiers that need to be mapped to gene symbols

# Therefore, we need to perform gene mapping
requires_gene_mapping = True

# Print the conclusion for clarity
print(f"Based on the observed identifiers which are numeric ('3', '5', '16', etc.), these are not human gene symbols.")
print(f"These appear to be probe IDs that require mapping to standard gene symbols.")
print(f"requires_gene_mapping = {requires_gene_mapping}")


Based on the observed identifiers which are numeric ('3', '5', '16', etc.), these are not human gene symbols.
These appear to be probe IDs that require mapping to standard gene symbols.
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'description', 'uniprot', 'gene ID', 'REFSEQ']
{'ID': ['3', '5', '16', '18', '20'], 'description': ['IL1B: (IL1B) INTERLEUKIN-1 BETA PRECURSOR (IL-1 BETA) (CATABOLIN).', 'IL2: (IL2 OR IL-2) INTERLEUKIN-2 PRECURSOR (IL-2) (T-CELL GROWTH FACTOR) (TCGF) (ALDESLEUKIN).', 'IL7: (IL7 OR IL-7) INTERLEUKIN-7 PRECURSOR (IL-7).', 'IL8_HUMAN: (IL8) INTERLEUKIN-8 PRECURSOR (IL-8) (CXCL8) (MONOCYTE-DERIVED NEUTROPHIL CHEMOTACTIC FACTOR) (MDNCF) (T-CELL CHEMOTACTIC FACTOR) (NEUTROPHIL-ACTIVATING PROTEIN 1) (NAP-1) (LYMPHOCYTE-DERIVED NEUTROPHIL-ACTIVATING FACTOR) (LYNAP) (PROTEIN 3-10C) (NEUTROPHIL-ACTIVATING FACTOR) (NAF) (GRANULOCYTE CHEMOTACTIC PROTEIN 1) (GCP-1) (EMOCTAKIN).', 'IL9: (IL9) INTERLEUKIN-9 PRECURSOR (IL-9) (T-CELL GROWTH FACTOR P40) (P40 CYTOKINE).'], 'uniprot': ['sp|P01584,sp|Q96HE5,sp|Q9UCT6,sp|Q7RU01', 'sp|P01585,tr|Q13169,sp|P60568', 'sp|P13232', 'sp|P10145,sp|Q9C077,sp|Q96RG6,sp|Q6FGF6,sp|Q6LAE6', 'sp|P15248'], 'gene 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns contain gene IDs and gene symbols
# From the preview, the 'ID' column matches the numeric identifiers in gene_data
# The 'description' column contains gene symbols at the start (e.g., "IL1B:")

print("Starting gene identifier mapping...")

# Function to extract gene symbols from the description field
def extract_gene_symbol(description):
    if not isinstance(description, str):
        return None
    # Extract text before the colon
    match = re.match(r'^([^:]+):', description)
    if match:
        return match.group(1).strip()
    return None

# Add a column with extracted gene symbols
gene_annotation['Gene_Symbol'] = gene_annotation['description'].apply(extract_gene_symbol)

# Preview the extracted gene symbols
print("Sample of extracted gene symbols:")
print(gene_annotation[['ID', 'Gene_Symbol']].head())

# 2. Create gene mapping dataframe
print("Creating gene mapping dataframe...")
mapping_df = gene_annotation[['ID', 'Gene_Symbol']].dropna()
mapping_df = mapping_df.rename(columns={'Gene_Symbol': 'Gene'})

# 3. Apply gene mapping to convert probe-level data to gene expression data
print("Converting probe data to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print shape and preview of the gene expression data
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols in mapped data:")
print(gene_data.index[:10].tolist())

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Starting gene identifier mapping...
Sample of extracted gene symbols:


   ID Gene_Symbol
0   3        IL1B
1   5         IL2
2  16         IL7
3  18   IL8_HUMAN
4  20         IL9
Creating gene mapping dataframe...
Converting probe data to gene expression data...
Gene expression data shape after mapping: (1369, 150)
First few gene symbols in mapped data:
['ABCA12', 'ABI3BP', 'ABME', 'ACADVL', 'ACP5', 'ACPP', 'ACSL3', 'ACTA2', 'ACTB', 'ACTG1']


Gene expression data saved to ../../output/preprocess/Eczema/gene_data/GSE63741.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using NCBI Gene database information
print("Normalizing gene symbols...")
try:
    # Load the gene data if needed
    if 'gene_data' not in locals() or gene_data is None:
        gene_data = pd.read_csv(out_gene_data_file, index_col=0)
        
    # Normalize gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"Sample of normalized gene symbols: {normalized_gene_data.index[:10].tolist()}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")

# 2. Load the clinical data that was saved in step 2
print("\nLoading clinical data...")
try:
    clinical_features = pd.read_csv(out_clinical_data_file, index_col=0)
    print(f"Clinical features shape: {clinical_features.shape}")
    print("Clinical features preview:")
    print(preview_df(clinical_features))
    
    # 3. Link clinical and genetic data
    print("\nLinking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
        print(linked_data.iloc[:5, :5])
    else:
        print(linked_data)
    
    # 4. Handle missing values
    print("\nHandling missing values...")
    linked_data_clean = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")
    
    # 5. Check for bias in the dataset
    print("\nChecking for bias in dataset features...")
    is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)
    
    # 6. Conduct final quality validation
    note = "Dataset contains gene expression data from skin biopsies comparing different skin conditions including eczema (atopic dermatitis and contact eczema) against other conditions like psoriasis and healthy controls."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data_clean,
        note=note
    )
    
    # 7. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data_clean.to_csv(out_data_file, index=True)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset deemed not usable for associative studies. Linked data not saved.")
        
except Exception as e:
    print(f"Error processing data: {e}")
    # If processing fails, we should still validate the dataset status
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,  # We know trait data is available from step 2
        is_biased=True,  # Set to True to ensure it's not marked usable
        df=pd.DataFrame(),  # Empty dataframe since processing failed
        note=f"Failed to process data: {e}"
    )
    print("Dataset validation completed with error status.")

Normalizing gene symbols...


Gene data shape after normalization: (1259, 150)
Sample of normalized gene symbols: ['ABCA12', 'ABI1', 'ABI3BP', 'ACADVL', 'ACER1', 'ACER3', 'ACP3', 'ACP5', 'ACSL3', 'ACTA2']
Normalized gene data saved to ../../output/preprocess/Eczema/gene_data/GSE63741.csv

Loading clinical data...
Clinical features shape: (1, 150)
Clinical features preview:
{'GSM1556392': [0.0], 'GSM1556393': [0.0], 'GSM1556394': [0.0], 'GSM1556395': [0.0], 'GSM1556396': [0.0], 'GSM1556397': [0.0], 'GSM1556398': [0.0], 'GSM1556399': [0.0], 'GSM1556400': [0.0], 'GSM1556401': [0.0], 'GSM1556402': [0.0], 'GSM1556403': [0.0], 'GSM1556404': [0.0], 'GSM1556405': [0.0], 'GSM1556406': [0.0], 'GSM1556407': [0.0], 'GSM1556408': [0.0], 'GSM1556409': [0.0], 'GSM1556410': [0.0], 'GSM1556411': [0.0], 'GSM1556412': [0.0], 'GSM1556413': [0.0], 'GSM1556414': [0.0], 'GSM1556415': [0.0], 'GSM1556416': [0.0], 'GSM1556417': [0.0], 'GSM1556418': [0.0], 'GSM1556419': [0.0], 'GSM1556420': [0.0], 'GSM1556421': [0.0], 'GSM1556422': [0.0], 'G

Linked data shape after handling missing values: (150, 1260)

Checking for bias in dataset features...
Quartiles for 'Eczema':
  25%: 0.0
  50% (Median): 0.0
  75%: 0.0
Min: 0.0
Max: 0.0
The distribution of the feature 'Eczema' in this dataset is severely biased.



Dataset deemed not usable for associative studies. Linked data not saved.
